In [ ]:
!pip install qdrant-client --quiet

In [1]:
# Эта часть с bge m3 общая, надо перенести в модуль .py

import torch
from torch import nn
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

# Примеры
input_texts = [
    "Когда был спущен на воду первый миноносец «Спокойный»?",
    "Есть ли нефть в Удмуртии?",
    "Спокойный (эсминец)\nЗачислен в списки ВМФ СССР 19 августа 1952 года.",
    "Нефтепоисковые работы в Удмуртии были начаты сразу после Второй мировой войны в 1945 году и продолжаются по сей день. Добыча нефти началась в 1967 году."
]

device = "cuda" if torch.cuda.is_available() else "cpu"

# Загружаем модель и токенизатор
tokenizer = AutoTokenizer.from_pretrained("deepvk/USER-bge-m3", use_fast=True)
model = AutoModel.from_pretrained("deepvk/USER-bge-m3")
model.to(device)
model.eval()

# Включаем TF32 для всех float32 матричных умножений
torch.set_float32_matmul_precision('high')

# Опционально ускоряем на через compile
if hasattr(torch, 'compile'):
    model = torch.compile(model)


def model_encode(input_texts, batch_size = 64):
    all_embeddings = []

    for i in tqdm(range(0, len(input_texts), batch_size), desc="Encoding batches"):
        batch_texts = input_texts[i:i+batch_size]
        
        encoded_input = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            return_tensors='pt'
        ).to(device)
        
        attention_mask = encoded_input['attention_mask']

        with torch.no_grad():
            model_output = model(**encoded_input)
            token_embeddings = model_output[0]  # [batch, seq_len, hidden]

            # Mean pooling и маска внимания
            mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            summed = torch.sum(token_embeddings * mask, dim=1)
            summed_mask = torch.clamp(mask.sum(dim=1), min=1e-9)
            embeddings = summed / summed_mask

            # Нормализация
            embeddings = nn.functional.normalize(embeddings, p=2, dim=1)
            all_embeddings.append(embeddings.cpu())

    # Объединяем все батчи
    sentence_embeddings = torch.cat(all_embeddings, dim=0)
    return sentence_embeddings

test = model_encode(input_texts)
test.shape


Encoding batches: 100%|██████████| 1/1 [00:22<00:00, 22.33s/it]


torch.Size([4, 1024])

In [ ]:
import torch 
import json

# Загрузка эмбс 
embs = torch.load("embeddings.pt")
print(embs.shape) 

# Загрузка чанков (сабсэмпл)
with open("chunks_sub.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)
print(len(chunks))

torch.Size([20000, 1024])
20000


In [3]:
from qdrant_client import QdrantClient
from tqdm import tqdm

# Подключение к Qdrant
# Условно здесь нужен хост, но пока точки будут храниться в RAM
client = QdrantClient(":memory:")


In [4]:
from qdrant_client.models import VectorParams, PointStruct

# Создаём коллекцию 
client.create_collection(
    collection_name="my_collection",
    vectors_config= VectorParams(size=1024, distance="Cosine")
)


True

In [5]:
# Объекты

# Пример 
'''
points = [ 
    # Можно передавать метадату, например, ссылку или city
    PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
    PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
    PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
    PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
] 
'''

# Собираем эмбс в PointStruct
points = [
    PointStruct(id = i, vector = emb.tolist(), payload={"city": "Berlin"})
    for i, emb in tqdm(enumerate(embs))
]

# Добавляем данные
client.upsert(
    collection_name="my_collection",
    wait=True,
    points=points
)


9016it [00:00, 18120.14it/s]

20000it [00:01, 19489.35it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [6]:
# Поиск ближайших векторов

# model_encode работает медленно, нужно ускорять 
# почистить датасет (например, есть дубли)

# Пример эмбса, по которому ищем ближайших соседей
# query_vector = [0.4, 0.5, 0.5, 0.75]

input = 'Легкая женская рубашка, белая'
#input = 'Смартфон Xiaomi, черный, высокопроизводительный, хороший экран'

input_embedded = model_encode([input], batch_size = 1) 

# Убрать ось и превратить в список
query_vector = input_embedded[0].tolist()


# Ищем 20 ближайших векторов
search_result = client.query_points(
    collection_name="my_collection",
    query=query_vector,
    limit=20,
    with_payload=True  # для метаданных
)

found = []

for pt in search_result.points:
    print(f"ID: {pt.id}, Score: {pt.score}, Payload: {pt.payload}")
    found.append(pt.id) 


Encoding batches: 100%|██████████| 1/1 [00:20<00:00, 20.69s/it]

ID: 2632, Score: 0.7714044898794246, Payload: {'city': 'Berlin'}
ID: 3047, Score: 0.7678803993947425, Payload: {'city': 'Berlin'}
ID: 17072, Score: 0.7671377734275099, Payload: {'city': 'Berlin'}
ID: 13658, Score: 0.7671377734275099, Payload: {'city': 'Berlin'}
ID: 8730, Score: 0.7668255086645352, Payload: {'city': 'Berlin'}
ID: 4583, Score: 0.7668255086645352, Payload: {'city': 'Berlin'}
ID: 5648, Score: 0.7646647215912676, Payload: {'city': 'Berlin'}
ID: 1954, Score: 0.7562074425792121, Payload: {'city': 'Berlin'}
ID: 3961, Score: 0.7561840128745841, Payload: {'city': 'Berlin'}
ID: 4564, Score: 0.7538648868568862, Payload: {'city': 'Berlin'}
ID: 15133, Score: 0.748489626290521, Payload: {'city': 'Berlin'}
ID: 17970, Score: 0.7479757130549884, Payload: {'city': 'Berlin'}
ID: 19455, Score: 0.7441119664661353, Payload: {'city': 'Berlin'}
ID: 12051, Score: 0.7419306280840067, Payload: {'city': 'Berlin'}
ID: 6694, Score: 0.7410004035450786, Payload: {'city': 'Berlin'}
ID: 2449, Score: 0.7

In [7]:
# Отображаем найденные 

for i, idx in enumerate(found):
    print(f'------ Найденный {i + 1}')
    print(chunks[idx])
    print() 

------ Найденный 1
Название: Блузка женская белая
Категория: zhenshchinam
Подкатегория: Блузки и рубашки
Бренд: By Babur
Цвет: белый
Размер: 40

Описание: Элегантная белая блузка рубашка с длинным рукавом. Застежка на пуговицы, на спине кокетка, кружевной воротник. Приталенный силуэт образуется за счет вытачек на спинке. Модель традиционного кроя выглядит сдержанно и строго. Легкая ткань приятна для тела и практична в использовании. Рубашка прекрасно сидит на фигуре. Изящная базовая рубашка в элегантном стиле станет универсальной основой незабываемых образов. Она незаменима для офиса, деловых встреч и официальных мероприятий. Базовая рубашка сочетается с одеждой разного стиля. С юбкой-карандашом и туфлями-лодочками вы сможете создать строгий и сдержанный комплект для работы. С джинсами, уютным кардиганом и кедами получится более расслабленный и непринужденный вариант для отдыха. Эта блузка женская белая с длинным рукавом и кружевами незаменима для вашего гардероба. 

------ Найденный 2